In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import urllib.request
import os

# **네이버 웹툰 10개 장르의 웹툰 제목, 작가, 별점, 썸네일 정보 크롤링**

**웹툰 장르 리스트와 각 웹툰의 title_id 크롤링**

In [ ]:
#웹툰 장르 리스트
webtoon_genre=[]
url = 'https://comic.naver.com/webtoon/genre'
resp = requests.get(url)
soup = BeautifulSoup(resp.content,'lxml')
a_tags = soup.select('ul.spot a')

for a_tag in a_tags:
  a_tag['href'] = a_tag['href'].split('?')[1]
  webtoon_genre.append(a_tag['href'])
# print(webtoon_genre)
#['genre=episode', 'genre=omnibus', 'genre=story', 'genre=daily', 'genre=comic', 'genre=fantasy', 'genre=action', 'genre=drama', 'genre=pure', 'genre=sensibility', 'genre=thrill', 'genre=historical', 'genre=sports']

#모든 웹툰들의 타이틀아이디
webtoon_title_id = []

for genre in webtoon_genre:
  url = f'https://comic.naver.com/webtoon/genre?{genre}'
  resp = requests.get(url)
  soup = BeautifulSoup(resp.text,'lxml')
  html = soup.select('ul.img_list li dl dt a')
  for j in range(len(html)):
    webtoon_title_id.append(html[j]['href'])
# print(webtoon_title_id)
len(webtoon_title_id)  

3912

In [ ]:
#각 웹툰의 마지막 페이지와 최신화 크롤링 코드 / 썸네일을 가져올때 사용됨
page_size=[]
total_hwa = []
for i in tqdm(webtoon_title_id):
  url =f"https://comic.naver.com{i}"
  res = requests.get(url)
  soup = BeautifulSoup(res.content, 'lxml')

  page_tag =soup.select("table.viewList tr td.title  a")
  total_hwa.append(int(page_tag[0]["href"].split("&")[1].strip("no=")))
  page_size.append(int(page_tag[0]["href"].split("&")[1].strip("no="))//10+1)

len(total_hwa)

  0%|          | 0/3912 [00:00<?, ?it/s]

3912

In [ ]:
len(total_hwa),len(page_size)

(3912, 3912)

## **각 웹툰들의 종합정보 크롤링** 
**(실제로 다시할때는 양이 많기때문에 각 장르별로 나눠하는걸 추천한다.)**

**앞서 구한 title_id 리스트와 각 웹툰의 마지막 페이지 리스트를 통해서 크롤링**

In [ ]:
dic_webtoon={
    '제목' : [],
    '장르' : [],
    '작가':[],
    '별점':[],
    '이미지' :[]
}

for x in tqdm(range(len(webtoon_title_id))):
  for i in tqdm(range(page_size[x])):

    url = f'https://comic.naver.com/{webtoon_title_id[x]}&page={i+1}'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text,'lxml')
    table= soup.select('table.viewList')
    thumbnail = table[0].find_all('img')
    rating = soup.select('div.rating_type')
    author = soup.select('span.wrt_nm')
    title= soup.select('span.title')

    n=0
    #for j in range(len(thumbnail))
    for j in thumbnail:
      if('thumbnail' in j['src']):
        dic_webtoon['이미지'].append(j['src'])
        dic_webtoon['별점'].append(rating[n].text.split('\n')[-2])
        dic_webtoon['제목'].append(title[0].text)
        dic_webtoon['작가'].append(author[0].text.strip('\n\t\t\t\t\t\t\t'))
        dic_webtoon['장르'].append('action')
        n+=1

## **크롤링 해온 웹툰들의 정보들을 데이터프레임화**

In [ ]:
#웹툰들의 정보 데이터프레임화
import pandas as pd
df_webtoon = pd.DataFrame(dic_webtoon).reset_index(drop=True)

df_webtoon

In [ ]:
#웹툰 제목 리스트
name = []
for i in df_webtoon['제목']:
  if(i not in name):
    name.append(i)
name

In [ ]:
#웹툰 타이틀아이디 번호부분 추출
webtoon_id = []
for i in webtoon_title_id:
  webtoon_id.append(i.split('=')[-1])

In [ ]:
#이미지의 파일의 경로 리스트
file_name = []
for i in tqdm(range(len(name))):
  now_webtoon = df_webtoon[df_webtoon['제목'] == name[i]]
  for j in tqdm(range(len(now_webtoon))):
    file_name.append(f"./img/action/{j+1}_action_{webtoon_id[i]}.jpg")

In [ ]:
#썸네일 다운로드
for i in tqdm(range(len(file_name))):
  
  urllib.request.urlretrieve(df_webtoon['이미지'][i], file_name[i])

In [ ]:
!zip -r /content/img.zip /content/img
#/content/img.zip :만들 zip 파일의 이름과 저장위치
#/content/img : 압출할 폴더

In [ ]:
#데이터프레임에 넣을 이미지 경로
image_name = []
for i in range(len(file_name)):
  src = './img/'+file_name[i].split('/')[-1]
  image_name.append(src)
image_name

In [ ]:
#썸네일이 들어있는 링크가 아닌 이미지가 있는 경로로 바꿔주는 작업
for i in range(len(image_name)):
  df_webtoon['이미지'][i] = image_name[i]

In [ ]:
# df_webtoon = df_webtoon.drop('Unnamed: 0', axis=1)
# df_webtoon

In [ ]:
df_webtoon.to_csv('네이버웹툰 장르별통합정보.csv', encoding='utf-8-sig')